In [21]:
import torch
import random
import copy
import itertools
import numpy as np
from sklearn.metrics import classification_report
import utils

from LIM_deep_neural_classifier import LIMDeepNeuralClassifier
import dataset_semantic
from trainer import LIMTrainer

In [22]:
utils.fix_random_seeds()

In [23]:
PATH = "basemodel"


X, y = dataset_semantic.generate_data(one_hot=False)

In [24]:
print(len(X))

84


In [25]:
print(len(y))
print(len(y[0]))

84
53


In [26]:
embedding_dim = 256
VOCAB_SIZE = 25
LIM = LIMDeepNeuralClassifier(
    hidden_dim=512, 
    hidden_activation=torch.nn.ReLU(), 
    #hidden_activation=lambda x: x, 
    num_layers=2,
    input_dim=embedding_dim*2,
    n_classes=53,
   embedding_size_and_dim=(VOCAB_SIZE,embedding_dim)
    )
LIM_trainer = LIMTrainer(
    LIM,
    warm_start=True,
    max_iter=10000,
    batch_size=2,
    l2_strength=0.0,
    n_iter_no_change=10000,
    shuffle_train=True,
    eta=0.0001)

In [27]:

_ = LIM_trainer.fit(X, y)

Finished epoch 10000 of 10000; error is 115.74587672948837

In [28]:
preds = LIM_trainer.predict_logits(X).cpu()

In [29]:
print(preds.shape)

torch.Size([84, 53])


In [30]:
print(dataset_semantic.classwise_report(y, preds, classes="all"))

C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anac

CLASSIFICATION FOR CLASS:pine
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        83
         1.0       0.00      0.00      0.00         1

    accuracy                           0.99        84
   macro avg       0.49      0.50      0.50        84
weighted avg       0.98      0.99      0.98        84


CLASSIFICATION FOR CLASS:oak
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        83
         1.0       0.00      0.00      0.00         1

    accuracy                           0.99        84
   macro avg       0.49      0.50      0.50        84
weighted avg       0.98      0.99      0.98        84


CLASSIFICATION FOR CLASS:maple
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        83
         1.0       0.00      0.00      0.00         1

    accuracy                           0.99        84
   macro avg       0.49      0.50   

C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anac

In [31]:
torch.save(LIM_trainer.model.state_dict(), PATH)

In [32]:
dataset =dataset_semantic.generate_data_plants_have_roots_and_animals_can_move_and_have_skin(one_hot=False)
iit_data = tuple(dataset[2:5])
X_base_train, y_base_train, X_sources_train, y_IIT_train, interventions, blackout_classes = dataset

In [33]:
print(sum(y[:,22]))
print(sum(y_IIT_train[:,22]))
dataset_semantic.class_to_index("animals")

tensor(13.)
tensor(1092.)


22

In [34]:
LIM_trainer.model.set_analysis_mode(True)
LIM_trainer.max_iter = 10
LIM_trainer.blackoutclasses = blackout_classes
LIM_trainer.l2_strength = 0.0
LIM_trainer.shuffle_train = False
LIM_trainer.eta = 0.0001

In [35]:

id_to_coords = {
    0: [{"layer": 1, "start": 0, "end": int(0.25*embedding_dim)}]
}
_ = LIM_trainer.fit(
    X_base_train, 
    y_base_train,    
    iit_data=iit_data,
    intervention_ids_to_coords=id_to_coords)

Finished epoch 10 of 10; error is 102688195.53674316

In [36]:
base_preds = LIM_trainer.predict_logits(X_base_train.cpu())
print(dataset_semantic.classwise_report(y_base_train.cpu(), base_preds.cpu()))

C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anac

C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anac

C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anac

CLASSIFICATION FOR CLASS:pine
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6972
         1.0       0.00      0.00      0.00        84

    accuracy                           0.99      7056
   macro avg       0.49      0.50      0.50      7056
weighted avg       0.98      0.99      0.98      7056


CLASSIFICATION FOR CLASS:oak
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6972
         1.0       0.00      0.00      0.00        84

    accuracy                           0.99      7056
   macro avg       0.49      0.50      0.50      7056
weighted avg       0.98      0.99      0.98      7056


CLASSIFICATION FOR CLASS:maple
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6972
         1.0       0.00      0.00      0.00        84

    accuracy                           0.99      7056
   macro avg       0.49      0.50   

C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anaconda3\envs\bigkid\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\attic\anac

In [37]:
IIT_preds = LIM_trainer.iit_predict_logits(X_base_train.cpu(),
                                    [X_sources_train[0].cpu()],
                                    interventions.cpu(),
                                    id_to_coords)
print(dataset_semantic.classwise_report(y_IIT_train.cpu(), IIT_preds.cpu(), classes=["animals","plants","skin","move","roots"]))

CLASSIFICATION FOR CLASS:animals
              precision    recall  f1-score   support

         0.0       0.86      1.00      0.92      5964
         1.0       0.98      0.12      0.21      1092

    accuracy                           0.86      7056
   macro avg       0.92      0.56      0.57      7056
weighted avg       0.88      0.86      0.81      7056


CLASSIFICATION FOR CLASS:plants
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      6384
         1.0       0.48      0.03      0.06       672

    accuracy                           0.90      7056
   macro avg       0.69      0.52      0.51      7056
weighted avg       0.87      0.90      0.87      7056


CLASSIFICATION FOR CLASS:skin
              precision    recall  f1-score   support

         0.0       0.85      0.99      0.92      5964
         1.0       0.53      0.07      0.13      1092

    accuracy                           0.85      7056
   macro avg       0.69      0.

In [38]:
for layer in [0,1]:    
    for int_size in [0.5,0.25,0.75, 0.0625, 0.125,]:
        id_to_coords = {
        0: [{"layer": layer, "start": 0, "end": int(int_size*embedding_dim*2)}]
        }
        LIM_trainer.model.load_state_dict(torch.load(PATH))
        _ = LIM_trainer.fit(
        X_base_train, 
        y_base_train,    
        iit_data=iit_data,
        intervention_ids_to_coords=id_to_coords)
        IIT_preds = LIM_trainer.iit_predict_logits(X_base_train.cpu(),
                                    [X_sources_train[0].cpu()],
                                    interventions.cpu(),
                                    id_to_coords)
        print(f"\n\nLayer:{layer} {int_size}")
        print(dataset_semantic.classwise_report(y_IIT_train.cpu(), IIT_preds.cpu(), classes=["animals","plants","skin","move","roots"]))

Finished epoch 8 of 10; error is 385672.86227130896

KeyboardInterrupt: 

## 